<a href="https://colab.research.google.com/github/MunJinSeo/MachineLearning/blob/main/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5(AAI107)_Team13_%EB%89%B4%EC%8A%A4%ED%81%AC%EB%A1%A4%EB%A7%81_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 팍스넷, 네이버 증권 뉴스 크롤링
# https://github.com/CSJ516/Stock_Prediction_project

import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse
from datetime import datetime

In [ ]:
# all_data 대상일자 추출
#target_date =  all_data.index.strftime('%Y%m%d').copy() # pd.DataFrame(all_data.index).copy()
target_date = {"Date":['20220601','20220602','20220603']}
df_target_date = pd.DataFrame(target_date)
print(df_target_date)
print(type(df_target_date))
print(df_target_date["Date"].tail(1))

       Date
0  20220601
1  20220602
2  20220603
<class 'pandas.core.frame.DataFrame'>
2    20220603
Name: Date, dtype: object


In [ ]:
def paxnet_news_title(date_list):
  # 팍스넷 > 뉴스 > 많이본뉴스 > 증권
  paxnet_url = 'http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo={}&genDate={}&objId=N4667'
  #paxnet_url = 'http://www.paxnet.co.kr/news/much?newsSetId=4667&objId=N4667'
  
  webheader = {
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
  }

  paxnet_result = []
  paxnet_error_cnt = 0
  
  for t_date in date_list:
    for pagecnt in range(1, 2):
      target_url = paxnet_url.format(pagecnt, t_date)
      print(target_url)
      responseData = requests.get(target_url, headers=webheader)
      if responseData.status_code == 200:
        resParse = BeautifulSoup(responseData.text)
        title_list = resParse.select('ul.thumb-list li')
        for ttt in title_list:
          #print(ttt)
          try:
            news_title = ttt.select_one('dl.text > dt').text.strip()
            paxnet_result.append([news_title])
          except:
            paxnet_error_cnt += 1
  return paxnet_result

In [ ]:
paxnet_news_rs = paxnet_news_title(df_target_date["Date"].tail(1)) # 샘플로 최근 1일자만 추출해봄
#paxnet_news_rs = paxnet_news_title(df_target_date["Date"].head(1)) # 샘플로 최초 1일자만 추출해봄
print("paxnet_news_rs length == ", len( paxnet_news_rs ) )

http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo=1&genDate=20220603&objId=N4667
paxnet_news_rs length ==  20


In [ ]:
paxnet_title_df = pd.DataFrame(paxnet_news_rs, columns=['뉴스제목'])
#paxnet_title_df['주가변동'] = 0  # all_data["Class"] 값으로 맵핑 필요 (해당 일자에 맞춰 주가 상승, 보합, 하락에 따라 분류)
paxnet_title_df

,뉴스제목
0,국내 환율 마감시황
1,FTSE100(영국) : Platinum Jubilee Bank Holiday로 휴장
2,국내 채권금리 동향(마감)
3,항셍지수(홍콩) : 단오절(Tuen Ng Festival)로 휴장
4,뉴욕증시 마감시황
5,미국 주요기업 주가등락률(마감)
6,미국 주요 ETF 등락률
7,미국시장 테마시황
8,[뉴욕증시] 테슬라·메타 급등에 일제히 상승…나스닥 2.69% ↑
9,국내외 주요 마감지수


In [ ]:
def naver_news_title(date_list):
  # 네이버 > 금융 > 많이본뉴스
  naver_url = 'https://finance.naver.com/news/news_list.naver?mode=RANK&page={}&date={}'
  
  webheader = {
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
  }

  naver_result = []
  naver_error_cnt = 0

  for t_date in date_list:
    for pagecnt in range(1, 2):
      target_url = naver_url.format(pagecnt, t_date)
      print(target_url)
      responseData = requests.get(target_url, headers=webheader)
      if responseData.status_code == 200:
        resParse = BeautifulSoup(responseData.text)
        title_list = resParse.select('ul.simpleNewsList li')
        for ttt in title_list:
          #print("-----------------\n")
          #print(ttt)
          try:
            #news_title = ttt.select_one('li').text.strip()
            news_title = ttt.select_one('a').text.strip()
            #print("news_title==",news_title)
            naver_result.append([news_title])
          except:
            naver_error_cnt += 1
  return naver_result

In [ ]:
naver_news_rs = naver_news_title(df_target_date["Date"].tail(1)) # 샘플로 최근 1일자만 추출해봄
print("naver_news_rs length == ", len( naver_news_rs ) )

https://finance.naver.com/news/news_list.naver?mode=RANK&page=1&date=20220603
naver_news_rs length ==  25


In [ ]:
naver_title_df = pd.DataFrame(naver_news_rs, columns=['뉴스제목'])
#naver_title_df['주가변동'] = 0  # all_data["Class"] 값으로 맵핑 필요 (해당 일자에 맞춰 주가 상승, 보합, 하락에 따라 분류)
naver_title_df

,뉴스제목
0,"델리오, 첫 콜라보 NFT ‘클레이다이노 블루베이 에디션’ 경매.."
1,"[특징주] 한국조선해양, 수주 계약 취소에도… 주가는 2% 상승"
2,"거래소, ITX-AI 상장적격성 실질심사 사유 추가"
3,[특징주] 인천공항 국제선 정상화에 항공주 강세
4,[특징주] 우크라 외교차관 방한에 재건주 '강세'… 서암기계공업..
5,"[특징주] 청담글로벌, 코스닥 상장 첫날 공모가 대비 27%↑(.."
6,"금융위, 중기 특화 증권사 7곳 지정…케이프·DS 포함"
7,"에스티아이, 잉크젯 장비 수주 계약 체결"
8,"[특징주]디와이파워, 재건 중장비 '근육' 담당…우크라이나 재건.."
9,"상승장서 소외된 건설주, 선거 뒤 탄력받을까…증권가는 ""사라"""


In [ ]:
def naver_market_news_title(date_list):
  # 네이버 > 금융 > 뉴스포커스(시황/전망)
  naver_market_url = 'https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=401&page={}&date={}'
  
  webheader = {
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
  }

  naver_market_result = []
  naver_market_error_cnt = 0

  for t_date in date_list:
    for pagecnt in range(1, 2):
      target_url = naver_market_url.format(pagecnt, t_date)
      print(target_url)
      responseData = requests.get(target_url, headers=webheader)
      if responseData.status_code == 200:
        resParse = BeautifulSoup(responseData.text)
        title_list = resParse.select('dd.articleSubject')
        for ttt in title_list:
          #print("-----------------\n")
          #print(ttt)
          try:
            #news_title = ttt.select_one('li').text.strip()
            news_title = ttt.select_one('a').text.strip()
            #print("news_title==",news_title)
            naver_market_result.append([news_title])
          except:
            naver_market_error_cnt += 1
  return naver_market_result

In [ ]:
naver_market_news_rs = naver_market_news_title(df_target_date["Date"].tail(1)) # 샘플로 최근 1일자만 추출해봄
print("naver_market_news_rs length == ", len( naver_market_news_rs ) )

https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=401&page=1&date=20220603
naver_market_news_rs length ==  18


In [ ]:
naver_market_title_df = pd.DataFrame(naver_market_news_rs, columns=['뉴스제목'])
#naver_market_title_df['주가변동'] = 0  # all_data["Class"] 값으로 맵핑 필요 (해당 일자에 맞춰 주가 상승, 보합, 하락에 따라 분류)
naver_market_title_df

,뉴스제목
0,"한국투자증권, 미국 부동산 투자자문 서비스 개시"
1,[표]유가증권·코스닥 투자주체별 매매동향(5월 31일-최종치)
2,'목표가 93만원→48만원'…삼성SDI 혹평한 씨티증권에 '소나기 반박'
3,[표] 주간 수익률 베스트&워스트
4,[표] 주요 증시 지표 (6월 3일)
5,[ETF줌인] 명품 투자 ETF…면세점 매출 회복 수혜 기대
6,"대신증권, '비상장주식 안심거래 서비스' 실시"
7,[장외주식] 쓰리디팩토리 나흘 연속 상승
8,[표]코스피200지수 옵션 시세(6월 3일)
9,[표]코스피200지수·국채·달러 선물 시세(6월 3일)


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
#naver_market_title_df
#naver_title_df
#paxnet_title_df
#df_list3 = list(naver_title_df["뉴스제목"])
df_list = [
"현대바이오, '폴리탁셀' 코로나19 치료 가능성에 19% 급등" #Positive
,"이수화학, 3분기 영업익 176억…전년比 80%↑" #Positive
,"GKL, 7년 만에 두 자릿수 매출성장 예상" #Positive
,"위지윅스튜디오, 콘텐츠 활약에 사상 첫 매출 1000억원 돌파" #Positive
,"삼성전자, 2년 만에 인도 스마트폰 시장 점유율 1위 '왕좌 탈환'" #Positive
,"영화관株 '코로나 빙하기' 언제 끝나나… CJ CGV 올 4000억 손실 날수도" #Negative
,"C쇼크에 멈춘 흑자비행…대한항공 1분기 영업적자 566억" #Negative
,"'1000억대 횡령·배임' 최신원 회장 구속… SK네트웍스 경영 공백 방지 최선" #Negative
,"부품 공급 차질에…기아차 광주공장 전면 가동 중단" #Negative
,"현대제철, 지난해 영업익 3,313억원···전년比 67.7% 감소" #Negative
]
#df_list.append(df_list2)


In [ ]:
#https://github.com/snunlp/KR-FinBert-SC

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
#tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")
#model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert",num_labels=3)

In [ ]:
#tokenize text to be sent to model
inputs = tokenizer(df_list
                   , max_length=200
                   #, padding = True
                   , truncation = True
                   , return_tensors='pt'
                   ,add_special_tokens=True
                   ,return_attention_mask=True
                   ,padding='max_length'
)
outputs = model(**inputs)

In [ ]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
#predictions2 = torch.nn.functional.softmax(outputs.logits, dim=1)

In [ ]:
predictions

In [ ]:
#label_dict = {0: 'negative', 1: 'neutral', 2: 'positive'}
print(model.config.id2label)

{0: 'negative', 1: 'neutral', 2: 'positive'}


In [ ]:
negative = predictions[:, 0].tolist()
neutral = predictions[:, 1].tolist()
positive = predictions[:, 2].tolist()

table = {'Headline':df_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral
         ,"label_Positive" : [ 1 if cc > 0.5 else 0 for cc in positive ] # 분류 : 상승 1, 보함 0 , 하락 -1
         }
      
df2 = pd.DataFrame(table, columns = table.keys() )

In [ ]:
# 나중에 일자별로 뉴스를 묶을때 사용
# DataFrame.groupby("컬럼명").mean()

In [ ]:
df2

,Headline,Positive,Negative,Neutral,label_Positive
0,"현대바이오, '폴리탁셀' 코로나19 치료 가능성에 19% 급등",0.999870,0.000034,0.000096,1
1,"이수화학, 3분기 영업익 176억…전년比 80%↑",0.980786,0.019011,0.000203,1
2,"GKL, 7년 만에 두 자릿수 매출성장 예상",0.999851,0.000029,0.000121,1
3,"위지윅스튜디오, 콘텐츠 활약에 사상 첫 매출 1000억원 돌파",0.999883,0.000041,0.000077,1
4,"삼성전자, 2년 만에 인도 스마트폰 시장 점유율 1위 '왕좌 탈환'",0.999881,0.000045,0.000074,1
5,영화관株 '코로나 빙하기' 언제 끝나나… CJ CGV 올 4000억 손실 날수도,0.000088,0.999813,0.000098,0
6,C쇼크에 멈춘 흑자비행…대한항공 1분기 영업적자 566억,0.000093,0.999813,0.000094,0
7,'1000억대 횡령·배임' 최신원 회장 구속… SK네트웍스 경영 공백 방지 최선,0.000115,0.999686,0.000198,0
8,부품 공급 차질에…기아차 광주공장 전면 가동 중단,0.000071,0.999788,0.000141,0
9,"현대제철, 지난해 영업익 3,313억원···전년比 67.7% 감소",0.000078,0.999823,0.000100,0


In [ ]:
df2.groupby("label_Positive").mean()

,Positive,Negative,Neutral
label_Positive,,,
0,0.000089,0.999785,0.000126
1,0.996054,0.003832,0.000114
